In [ ]:
import torch
import torchaudio
from datasets import load_dataset, load_metric
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
import re

In [ ]:
test_dataset = load_dataset("mozilla-foundation/common_voice_11_0", "ta", split="test")


In [ ]:
#pip install datasets
!pip install jiwer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 33.6 MB/s eta 0:00:00


In [ ]:
wer = load_metric("wer")

In [ ]:
model="vasista22/whisper-tamil-large-v2"
processor = Wav2Vec2Processor.from_pretrained(model)
model = Wav2Vec2ForCTC.from_pretrained(model)


preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/832 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

...

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

You are using a model of type whisper to instantiate a model of type wav2vec2. This is not supported for all configurations of models and can yield errors.


pytorch_model.bin:   0%|          | 0.00/6.17G [00:00<?, ?B/s]

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at vasista22/whisper-tamil-large-v2 and are newly initialized: ['encoder.layer_norm.bias', 'encoder.layer_norm.weight', 'encoder.layers.0.attention.k_proj.bias', 'encoder.layers.0.attention.k_proj.weight', 'encoder.layers.0.attention.out_proj.bias', 'encoder.layers.0.attention.out_proj.weight', 'encoder.layers.0.attention.q_proj.bias', 'encoder.layers.0.attention.q_proj.weight', 'encoder.layers.0.attention.v_proj.bias', 'encoder.layers.0.attention.v_proj.weight', 'encoder.layers.0.feed_forward.intermediate_dense.bias', 'encoder.layers.0.feed_forward.intermediate_dense.weight', 'encoder.layers.0.feed_forward.output_dense.bias', 'encoder.layers.0.feed_forward.output_dense.weight', 'encoder.layers.0.final_layer_norm.bias', 'encoder.layers.0.final_layer_norm.weight', 'encoder.layers.0.layer_norm.bias', 'encoder.layers.0.layer_norm.weight', 'encoder.layers.1.attention.k_proj.bias', 'encoder.layers.1.attention.k_pr

In [ ]:
model.to("cuda")

Wav2Vec2ForCTC(
  (wav2vec2): Wav2Vec2Model(
    (feature_extractor): Wav2Vec2FeatureEncoder(
      (conv_layers): ModuleList(
        (0): Wav2Vec2GroupNormConvLayer(
          (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,), bias=False)
          (activation): GELUActivation()
          (layer_norm): GroupNorm(512, 512, eps=1e-05, affine=True)
        )
        (1-4): 4 x Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
          (activation): GELUActivation()
        )
        (5-6): 2 x Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(2,), stride=(2,), bias=False)
          (activation): GELUActivation()
        )
      )
    )
    (feature_projection): Wav2Vec2FeatureProjection(
      (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (projection): Linear(in_features=512, out_features=768, bias=True)
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder)

In [ ]:
!pip install jiwer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 44.2 MB/s eta 0:00:00


In [ ]:
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\“]'
resampler = torchaudio.transforms.Resample(48_000, 16_000)

In [ ]:
def speech_file_to_array_fn(batch):
    batch["sentence"] = re.sub(chars_to_ignore_regex, '', batch["sentence"]).lower()
    speech_array, sampling_rate = torchaudio.load(batch["path"])
    batch["speech"] = resampler(speech_array).squeeze().numpy()
    return batch

In [ ]:
def speech_file_to_array_fn(batch):
    batch["sentence"] = re.sub(chars_to_ignore_regex, '', batch["sentence"]).lower()
    speech_array, sampling_rate = torchaudio.load(batch["path"])
    # Add an extra dimension for batching
    batch["speech"] = resampler(speech_array).squeeze().numpy()[None]
    return batch

In [ ]:
test_dataset = test_dataset.map(speech_file_to_array_fn)


Map:   0%|          | 0/11815 [00:00<?, ? examples/s]

In [ ]:
def evaluate(batch):
    inputs = processor(batch["speech"], sampling_rate=16_000, return_tensors="pt", padding=True)

    with torch.no_grad():
        logits = model(inputs.input_values.to("cuda"), attention_mask=inputs.attention_mask.to("cuda")).logits

    pred_ids = torch.argmax(logits, dim=-1)
    batch["pred_strings"] = processor.batch_decode(pred_ids)
    return batch


In [ ]:
def evaluate(batch):
    inputs = processor(batch["speech"], sampling_rate=16_000, return_tensors="pt", padding=True)

    with torch.no_grad():
        logits = model(inputs.input_values.to("cuda"), attention_mask=inputs.attention_mask.to("cuda")).logits
        print(f"Logits shape: {logits.shape}")

    pred_ids = torch.argmax(logits, dim=-1)
    print(f"Predicted IDs shape: {pred_ids.shape}")

    batch["pred_strings"] = processor.batch_decode(pred_ids)
    print(f"Batch pred_strings length: {len(batch['pred_strings'])}")

    return batch


In [ ]:

result = test_dataset.map(evaluate, batched=True, batch_size=8)

Map:   0%|          | 0/11815 [00:00<?, ? examples/s]

ValueError: Unable to create tensor, you should probably activate padding with 'padding=True' to have batched tensors with the same length.

In [ ]:
print("WER: {:2f}".format(100 * wer.compute(predictions=result["pred_strings"], references=result["sentence"])))


NameError: name 'result' is not defined

In [ ]:
import torch
from transformers import pipeline
from datasets import load_dataset

# Load the Common Voice dataset (Tamil language split)
#dataset = load_dataset("mozilla-foundation/common_voice_11_0", "ta", split="test")

# Set device based on availability of CUDA
device = "cuda:0" if torch.cuda.is_available() else "cpu"

# Initialize the ASR pipeline with the specified model and chunk length
transcribe = pipeline(task="automatic-speech-recognition",
                      model="vasista22/whisper-tamil-large-v2",
                      chunk_length_s=30,
                      device=device)

# Set forced decoder IDs for Tamil transcription
transcribe.model.config.forced_decoder_ids = transcribe.tokenizer.get_decoder_prompt_ids(language="ta", task="transcribe")

# Function to transcribe audio
def transcribe_audio(batch):
    # Perform transcription on the audio array
    transcription = transcribe(batch["audio"]["array"])["text"]
    return {"transcription": transcription}

# Apply transcription function to the dataset
test_dataset = test_dataset.map(transcribe_audio)

# Display a few examples of the original sentences and their transcriptions
for i in range(5):
    print(f"Original: {test_dataset[i]['sentence']}")
    print(f"Transcription: {test_dataset[i]['transcription']}")
    print("-" * 80)


generation_config.json:   0%|          | 0.00/300 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
import torch
from transformers import pipeline
device = "cuda:0" if torch.cuda.is_available() else "cpu"
transcribe = pipeline(task="automatic-speech-recognition",
                      model="vasista22/whisper-tamil-large-v2",
                      chunk_length_s=30,
                      device=device)

# Set forced decoder IDs for Tamil transcription
transcribe.model.config.forced_decoder_ids = transcribe.tokenizer.get_decoder_prompt_ids(language="ta", task="transcribe")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
# Function to transcribe audio
def transcribe_audio(batch):
    # Perform transcription on the audio array
    transcription = transcribe(batch["audio"]["array"])["text"]
    return {"transcription": transcription}

# Apply transcription function to the dataset
test_dataset = test_dataset.map(transcribe_audio)
